In [1]:
import requests
import pandas as pd

KAKAO_API_KEY = "4f040348a11373f7f6d1cdae6778fd0f"  # 자신의 카카오 API 키로 교체

# 카카오맵 API를 이용해 지하철역의 위도와 경도 가져오기
def get_coordinates(station_name):
    url = "https://dapi.kakao.com/v2/local/search/keyword.json"
    headers = {"Authorization": f"KakaoAK {KAKAO_API_KEY}"}
    params = {
        "query": station_name,  # 지하철역 이름
        "category_group_code": "SW8",  # 지하철역 카테고리 코드
    }
    
    response = requests.get(url, headers=headers, params=params)
    data = response.json()
    
    if data.get("documents"):
        # 첫 번째 결과의 위도와 경도를 반환
        station = data["documents"][0]
        latitude = station["y"]
        longitude = station["x"]
        return latitude, longitude
    else:
        # 만약 결과가 없다면 None을 반환
        return None, None

# CSV 파일에서 지하철역 데이터 읽어오기
df = pd.read_csv('../Data/maps/subway_stations.csv')  # 파일 이름에 맞게 수정

# 각 지하철역에 대한 위도와 경도 가져오기
coordinates = []
for station_name in df['전철역명']:
    latitude, longitude = get_coordinates(station_name)
    coordinates.append((latitude, longitude))

# 데이터프레임에 좌표 추가
df['위도'] = [coord[0] for coord in coordinates]
df['경도'] = [coord[1] for coord in coordinates]

# 결과를 새로운 CSV 파일로 저장
df.to_csv('../Data/maps/subway_stations_with_coordinates.csv', index=False, encoding='utf-8')

print("CSV 파일에 좌표가 추가되었습니다.")


CSV 파일에 좌표가 추가되었습니다.


In [1]:
import pandas as pd

# 첫 번째 CSV 파일 경로
subway_data_path = '../Data/maps/seoul_subway.csv'  # 첫 번째 CSV 파일 경로
coordinates_data_path = '../Data/maps/subway_stations_with_coordinates.csv'  # 두 번째 CSV 파일 경로

# CSV 파일 읽기
subway_df = pd.read_csv(subway_data_path)
coordinates_df = pd.read_csv(coordinates_data_path)

# 좌표 추가를 위한 새로운 열 생성
subway_df['위도'] = None
subway_df['경도'] = None

# 역명에 따라 좌표 값 추가
for index, row in subway_df.iterrows():
    station_name = row['역명']  # 첫 번째 CSV에서 역명 열 이름
    matching_station = coordinates_df[coordinates_df['전철역명'] == station_name]  # 두 번째 CSV에서 전철역명 열 이름
    
    if not matching_station.empty:
        subway_df.at[index, '위도'] = matching_station['위도'].values[0]
        subway_df.at[index, '경도'] = matching_station['경도'].values[0]

subway_df.to_csv('../Data/maps/seoul_subway.csv', index=False, encoding='utf-8')


In [5]:
import pandas as pd
# 파일에서 데이터 읽기
df1 = pd.read_csv('../Data/maps/seoul_subway.csv')
df2 = pd.read_csv('../Data/maps/subway_stations.csv')

# 이름이 같은 역의 좌표 업데이트
for index, row in df2.iterrows():
    matching_rows = df1[df1['역명'] == row['역사명']]
    if not matching_rows.empty:
        df1.loc[matching_rows.index, ['위도', '경도']] = row['위도'], row['경도']
df1.to_csv('../Data/maps/updated_seoul_subway.csv', index=False, encoding='utf-8-sig')